In [2]:
import pandas as pd

In [9]:
df_motion = pd.read_parquet('/Users/narayanschuetz/tmp_data/MHC_healthkit/Wmzvhl88mFNfAXCDqy-dUR9Y.parquet')
df_workout = pd.read_parquet('/Users/narayanschuetz/tmp_data/healthkit_sleep.parquet')
df_sleep = pd.read_parquet('/Users/narayanschuetz/tmp_data/healthkit_sleep.parquet')

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from datetime import datetime

# Define the user_id to filter by
user_id = "Wmzvhl88mFNfAXCDqy-dUR9Y"

# Define the date range for filtering
start_date = '2017-08-02'
end_date = '2017-08-04'

# Convert to datetime objects for comparison
start_datetime = pd.to_datetime(start_date)
end_datetime = pd.to_datetime(end_date) + pd.Timedelta(days=1)  # Include the end date

# Filter each dataframe to only include data within the specified date range
# Convert timestamps to the same timezone-naive format before comparison
df_motion_filtered = df_motion[
    (df_motion['startTime'].dt.tz_localize(None) >= start_datetime) & 
    (df_motion['startTime'].dt.tz_localize(None) < end_datetime)
]

# Filter workout data by user_id and date range
df_workout_filtered = df_workout[
    (df_workout['healthCode'] == user_id) &
    (df_workout['startTime'].dt.tz_localize(None) >= start_datetime) & 
    (df_workout['startTime'].dt.tz_localize(None) < end_datetime)
]

# Filter sleep data by user_id and date range
df_sleep_filtered = df_sleep[
    (df_sleep['healthCode'] == user_id) &
    (df_sleep['startTime'].dt.tz_localize(None) >= start_datetime) & 
    (df_sleep['startTime'].dt.tz_localize(None) < end_datetime)
]

# Print the number of records in each filtered dataframe
print(f"Motion data from {start_date} to {end_date}: {len(df_motion_filtered)} records")
print(f"Workout data for user {user_id} from {start_date} to {end_date}: {len(df_workout_filtered)} records")
print(f"Sleep data for user {user_id} from {start_date} to {end_date}: {len(df_sleep_filtered)} records")

# Display the first few rows of each filtered dataframe
print("\nSample of filtered motion data:")
display(df_motion_filtered.head())

print("\nSample of filtered workout data:")
display(df_workout_filtered.head())

print("\nSample of filtered sleep data:")
display(df_sleep_filtered.head())


In [ ]:
df_motion_filtered.startTime_timezone_offset.value_counts()

In [ ]:
%matplotlib inline
plt.close()
# Filter heart rate data and plot the time series
heart_rate_mask = df_motion_filtered['type'] == 'HKQuantityTypeIdentifierHeartRate'
heart_rate_data = df_motion_filtered[heart_rate_mask]

# Convert to datetime index for time series plotting
heart_rate_data = heart_rate_data.set_index('startTime')
heart_rate_data = heart_rate_data.value#.resample('1min').mean()

# Plot the heart rate time series
plt.figure(figsize=(12, 6))
plt.plot(heart_rate_data, 'b-', marker='x', markersize=3)
plt.title('Heart Rate Time Series')
plt.xlabel('Time')
plt.ylabel('Heart Rate (count/s)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
heart_rate_data

In [ ]:
df_motion

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.plot(np.load("/var/folders/s5/9tx92s6n1r1g20vf5q0j7gxc0000gp/T/tmpw56anixc/2017-08-03.npy")[1][5].reshape(-1))
plt.show()


In [ ]:
heart_rate_data

In [ ]:
df_sleep_filtered.startTime_timezone_offset.value_counts()

In [ ]:
df_workout_filtered.startTime_timezone_offset.value_counts()

In [ ]:
df_motion.head()

In [ ]:
df_workout.head()

In [ ]:
df_sleep.head()

In [ ]:
# Let's visualize data for a single day from each dataframe

# First, let's convert the startTime to datetime if it's not already
df_motion['startTime'] = pd.to_datetime(df_motion['startTime'])
df_workout['startTime'] = pd.to_datetime(df_workout['startTime'])
df_sleep['startTime'] = pd.to_datetime(df_sleep['startTime'])

# Select a specific date for each dataframe
# For motion data
sample_date_motion = df_motion['startTime'].dt.date.min()  # Using the earliest date in the dataset
motion_day_data = df_motion[df_motion['startTime'].dt.date == sample_date_motion]

# For workout data
sample_date_workout = df_workout['startTime'].dt.date.min()  # Using the earliest date in the dataset
workout_day_data = df_workout[df_workout['startTime'].dt.date == sample_date_workout]

# For sleep data
sample_date_sleep = df_sleep['startTime'].dt.date.min()  # Using the earliest date in the dataset
sleep_day_data = df_sleep[df_sleep['startTime'].dt.date == sample_date_sleep]

# Display the data for each day
print(f"Motion data for {sample_date_motion}:")
display(motion_day_data)

print(f"\nWorkout data for {sample_date_workout}:")
display(workout_day_data)

print(f"\nSleep data for {sample_date_sleep}:")
display(sleep_day_data)

# Visualize the data
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import numpy as np

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))

# Plot motion data
if not motion_day_data.empty and 'value' in motion_day_data.columns:
    ax1.scatter(motion_day_data['startTime'], motion_day_data['value'], alpha=0.7)
    ax1.set_title(f'Motion Data for {sample_date_motion}')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    ax1.grid(True)

# Plot workout data
if not workout_day_data.empty:
    # For workout data, we might want to show different types
    if 'type' in workout_day_data.columns:
        workout_types = workout_day_data['type'].unique()
        for i, workout_type in enumerate(workout_types):
            type_data = workout_day_data[workout_day_data['type'] == workout_type]
            ax2.scatter(type_data['startTime'], [i] * len(type_data), label=workout_type, alpha=0.7)
    
    ax2.set_title(f'Workout Data for {sample_date_workout}')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Workout Type')
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    if 'type' in workout_day_data.columns and len(workout_types) > 0:
        ax2.set_yticks(range(len(workout_types)))
        ax2.set_yticklabels(workout_types)
    ax2.grid(True)
    ax2.legend()

# Plot sleep data
if not sleep_day_data.empty:
    # For sleep data, we might want to show sleep states
    if 'value' in sleep_day_data.columns:
        sleep_states = sleep_day_data['value'].unique()
        for i, state in enumerate(sleep_states):
            state_data = sleep_day_data[sleep_day_data['value'] == state]
            ax3.scatter(state_data['startTime'], [i] * len(state_data), label=state, alpha=0.7)
    
    ax3.set_title(f'Sleep Data for {sample_date_sleep}')
    ax3.set_xlabel('Time')
    ax3.set_ylabel('Sleep State')
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    if 'value' in sleep_day_data.columns and len(sleep_states) > 0:
        ax3.set_yticks(range(len(sleep_states)))
        ax3.set_yticklabels(sleep_states)
    ax3.grid(True)
    ax3.legend()

plt.tight_layout()
plt.show()
